强化学习设计用于创建能够有效解决特定任务的策略(policy)。
策略可以采取各种形式，从可微的映射(differentiable map)从观察空间到行动空间，到更多随机的方法，如对每个可能行动计算的值列表进行argmax操作。
策略可以是确定性(deterministic)的或随机的(stochastic)，可能包含复杂的元素，如递归神经网络（RNNs）或变换器(transformer)。

适应所有这些场景可能相当复杂。在这个简洁的教程中，我们将深入讨论TorchRL在策略(policy)构建方面的核心功能。
我们将主要关注在两个常见场景下的随机(stochastic)和Q值策略：使用多层感知器（MLP）或卷积神经网络（CNN）作为骨干。

## TensorDictModules 模块
与环境如何与TensorDict实例进行交互类似，这个用于表示策略policy和值函数(value function)的模块也做同样的事情。
核心思想很简单：将一个标准的模块(Module)（或任何其他函数）封装在一个类中，这个类知道需要读取(read)和传递(pass)给模块的条目(entry)，并记录分配条目的结果。
为了说明这一点，我们将使用最简单的策略：从观察空间到行动空间的确定性映射[类似一个Q表]。
为了最大的通用性，我们将使用一个LazyLinear模块，以及我们在上一教程中实例化的Pendulum环境。

In [1]:
import torch

from tensordict.nn import TensorDictModule
from torchrl.envs import GymEnv

env = GymEnv("Pendulum-v1")
module = torch.nn.LazyLinear(out_features=env.action_spec.shape[-1])
policy = TensorDictModule(
    module,
    in_keys=["observation"],
    out_keys=["action"],
)

D:\python39\lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


以上就是执行我们的策略所需的全部！
使用懒加载模块（lazy module）可以让我们绕过获取观察空间形状的需求，因为模块会自动决定它。
这个策略现在已经准备好在环境中运行了：

In [2]:
rollout = env.rollout(max_steps=10, policy=policy)
print(rollout)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, i

## Specialized wrappers 定制包装类
为了简化  Actor， ProbabilisticActor， ActorValueOperator 或者  ActorCriticOperator的整合。
Actor为in_keys和out_keys提供默认值，使得与许多常见环境的整合变得简单直接。[也就是说不用显示指出in-keys和out-keys是什么了]

In [3]:
from torchrl.modules import Actor

policy = Actor(module)
rollout = env.rollout(max_steps=10, policy=policy)
print(rollout)
print(policy.in_keys,policy.out_keys)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared=False),
        observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, i


## 网络
TorchRL还提供常规模块，可以在无需依赖于tensordict功能的情况下使用
。你会遇到的两个最常见的网络是MLP和ConvNet（CNN）模块。我们可以用这些模块中的一个替换我们的策略模块：

In [4]:
from torchrl.modules import MLP

module = MLP(
    out_features=env.action_spec.shape[-1],
    num_cells=[32, 64],
    activation_class=torch.nn.Tanh,
)
policy = Actor(module)
rollout = env.rollout(max_steps=10, policy=policy)

D:\python39\lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


## 这里要搞懂PPO （Proximal Policy Optimization）的原理 [论文地址](https://arxiv.org/abs/1707.06347)
像PPO这样的策略优化（policy-optimization）最先进实现要求策略policy是随机的 (stochastic)：
与上面的例子不同，模块现在编码了从观察空间到可能的动作分布(the possible actions)的参数空间( parameter space)的映射。
TorchRL通过将各种操作如从参数构建分布、从该分布采样和检索对数概率等归入一个类，来简化此类模块的设计。
这里，我们将利用三个组件创建一个依赖于常规正态分布的actor
1. An MLP backbone reading observations of size [3] and outputting a single tensor of size [2];

2. A NormalParamExtractor module that will split this output on two chunks, a mean and a standard deviation of size [1];

3. A ProbabilisticActor that will read those parameters as in_keys, create a distribution with them and populate our tensordict with samples and log-probabilities.

In [5]:
from tensordict.nn.distributions import NormalParamExtractor
from torch.distributions import Normal
from torchrl.modules import ProbabilisticActor

backbone = MLP(in_features=3, out_features=2)
extractor = NormalParamExtractor()
module = torch.nn.Sequential(backbone, extractor)
td_module = TensorDictModule(module, in_keys=["observation"], out_keys=["loc", "scale"])
policy = ProbabilisticActor(
    td_module,
    in_keys=["loc", "scale"],
    out_keys=["action"],
    distribution_class=Normal,
    return_log_prob=True,
)

rollout = env.rollout(max_steps=10, policy=policy)
print(rollout)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        loc: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                observation: Tensor(shape=torch.Size([10, 3]), device=cpu, dtype=torch.float32, is_shared=False),
                reward: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.float32, is_shared=False),
                terminated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False),
                truncated: Tensor(shape=torch.Size([10, 1]), device=cpu, dtype=torch.bool, is_shared=False)},
            batch_size=torch.Size([10]),
            device=cpu,
            is_shared